In [26]:
import pandas as pd
from ast import literal_eval
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [27]:
df = pd.read_csv('greenbasket-products.csv')

In [28]:
df.head()

,product_id,product_name,tags,category_name,description
0,1,Essence Mascara Lash Princess,"[""beauty"", ""mascara""]",Beauty,The Essence Mascara Lash Princess is a popular...
1,2,Eyeshadow Palette with Mirror,"[""beauty"", ""eyeshadow""]",Beauty,The Eyeshadow Palette with Mirror offers a ver...
2,3,Powder Canister,"[""beauty"", ""face powder""]",Beauty,The Powder Canister is a finely milled setting...
3,4,Red Lipstick,"[""beauty"", ""lipstick""]",Beauty,The Red Lipstick is a classic and bold choice ...
4,5,Red Nail Polish,"[""beauty"", ""nail polish""]",Beauty,The Red Nail Polish offers a rich and glossy r...


In [29]:
# lower case all fields
for column in df.columns[2:]:
    df[column] = df[column].apply(lambda x: x.lower())

In [30]:
df.head()

,product_id,product_name,tags,category_name,description
0,1,Essence Mascara Lash Princess,"[""beauty"", ""mascara""]",beauty,the essence mascara lash princess is a popular...
1,2,Eyeshadow Palette with Mirror,"[""beauty"", ""eyeshadow""]",beauty,the eyeshadow palette with mirror offers a ver...
2,3,Powder Canister,"[""beauty"", ""face powder""]",beauty,the powder canister is a finely milled setting...
3,4,Red Lipstick,"[""beauty"", ""lipstick""]",beauty,the red lipstick is a classic and bold choice ...
4,5,Red Nail Polish,"[""beauty"", ""nail polish""]",beauty,the red nail polish offers a rich and glossy r...


In [31]:
df.tags = df.tags.apply(lambda x: literal_eval(x))

In [32]:
df.tail()

,product_id,product_name,tags,category_name,description
189,190,IWC Ingenieur Automatic Steel,"[watches, luxury watches]",womens watches,the iwc ingenieur automatic steel watch is a d...
190,191,Rolex Cellini Moonphase,"[watches, luxury watches]",womens watches,the rolex cellini moonphase watch is a masterp...
191,192,Rolex Datejust Women,"[watches, luxury watches, women's watches]",womens watches,the rolex datejust women's watch is an iconic ...
192,193,Watch Gold for Women,"[watches, women's watches]",womens watches,the gold women's watch is a stunning accessory...
193,194,Women's Wrist Watch,"[watches, women's watches]",womens watches,the women's wrist watch is a versatile and fas...


In [33]:
df.tags = df.tags.apply(lambda x: [tag.replace(" ", "") for tag in x])

In [34]:
df.tail()

,product_id,product_name,tags,category_name,description
189,190,IWC Ingenieur Automatic Steel,"[watches, luxurywatches]",womens watches,the iwc ingenieur automatic steel watch is a d...
190,191,Rolex Cellini Moonphase,"[watches, luxurywatches]",womens watches,the rolex cellini moonphase watch is a masterp...
191,192,Rolex Datejust Women,"[watches, luxurywatches, women'swatches]",womens watches,the rolex datejust women's watch is an iconic ...
192,193,Watch Gold for Women,"[watches, women'swatches]",womens watches,the gold women's watch is a stunning accessory...
193,194,Women's Wrist Watch,"[watches, women'swatches]",womens watches,the women's wrist watch is a versatile and fas...


In [35]:
df.category_name = df.category_name.apply(lambda x: x.split())
df.description = df.description.apply(lambda x: x.split())

In [36]:
df.tail()

,product_id,product_name,tags,category_name,description
189,190,IWC Ingenieur Automatic Steel,"[watches, luxurywatches]","[womens, watches]","[the, iwc, ingenieur, automatic, steel, watch,..."
190,191,Rolex Cellini Moonphase,"[watches, luxurywatches]","[womens, watches]","[the, rolex, cellini, moonphase, watch, is, a,..."
191,192,Rolex Datejust Women,"[watches, luxurywatches, women'swatches]","[womens, watches]","[the, rolex, datejust, women's, watch, is, an,..."
192,193,Watch Gold for Women,"[watches, women'swatches]","[womens, watches]","[the, gold, women's, watch, is, a, stunning, a..."
193,194,Women's Wrist Watch,"[watches, women'swatches]","[womens, watches]","[the, women's, wrist, watch, is, a, versatile,..."


In [37]:
df['content'] = df['description'] + df['category_name'] + df['tags']

In [38]:
df.tail()

,product_id,product_name,tags,category_name,description,content
189,190,IWC Ingenieur Automatic Steel,"[watches, luxurywatches]","[womens, watches]","[the, iwc, ingenieur, automatic, steel, watch,...","[the, iwc, ingenieur, automatic, steel, watch,..."
190,191,Rolex Cellini Moonphase,"[watches, luxurywatches]","[womens, watches]","[the, rolex, cellini, moonphase, watch, is, a,...","[the, rolex, cellini, moonphase, watch, is, a,..."
191,192,Rolex Datejust Women,"[watches, luxurywatches, women'swatches]","[womens, watches]","[the, rolex, datejust, women's, watch, is, an,...","[the, rolex, datejust, women's, watch, is, an,..."
192,193,Watch Gold for Women,"[watches, women'swatches]","[womens, watches]","[the, gold, women's, watch, is, a, stunning, a...","[the, gold, women's, watch, is, a, stunning, a..."
193,194,Women's Wrist Watch,"[watches, women'swatches]","[womens, watches]","[the, women's, wrist, watch, is, a, versatile,...","[the, women's, wrist, watch, is, a, versatile,..."


In [39]:
training_df = df[['product_name', 'content']]

In [40]:
stemmer = PorterStemmer()
training_df['content'] = training_df['content'].apply(lambda words: [stemmer.stem(word) for word in words])

/var/folders/vb/czsq0w9s3kq5tfrvfgc52n3w0000gn/T/ipykernel_28306/2996104715.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_df['content'] = training_df['content'].apply(lambda words: [stemmer.stem(word) for word in words])


In [41]:
training_df['content'] = training_df['content'].apply(lambda x: " ".join(x))

/var/folders/vb/czsq0w9s3kq5tfrvfgc52n3w0000gn/T/ipykernel_28306/1871890271.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_df['content'] = training_df['content'].apply(lambda x: " ".join(x))


In [42]:
training_df

,product_name,content
0,Essence Mascara Lash Princess,the essenc mascara lash princess is a popular ...
1,Eyeshadow Palette with Mirror,the eyeshadow palett with mirror offer a versa...
2,Powder Canister,the powder canist is a fine mill set powder de...
3,Red Lipstick,the red lipstick is a classic and bold choic f...
4,Red Nail Polish,the red nail polish offer a rich and glossi re...
...,...,...
189,IWC Ingenieur Automatic Steel,the iwc ingenieur automat steel watch is a dur...
190,Rolex Cellini Moonphase,the rolex cellini moonphas watch is a masterpi...
191,Rolex Datejust Women,the rolex datejust women' watch is an icon tim...
192,Watch Gold for Women,the gold women' watch is a stun accessori that...


In [43]:
cv = CountVectorizer(max_features=5000, stop_words='english')
vectors = cv.fit_transform(training_df['content']).toarray()

vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [44]:
similarity = cosine_similarity(vectors)

similarity

array([[1.        , 0.13116517, 0.17611749, ..., 0.        , 0.        ,
        0.        ],
       [0.13116517, 1.        , 0.17902872, ..., 0.0285133 , 0.02961744,
        0.03178209],
       [0.17611749, 0.17902872, 1.        , ..., 0.06125639, 0.03181424,
        0.03413944],
       ...,
       [0.        , 0.0285133 , 0.06125639, ..., 1.        , 0.60803351,
        0.70684542],
       [0.        , 0.02961744, 0.03181424, ..., 0.60803351, 1.        ,
        0.56478249],
       [0.        , 0.03178209, 0.03413944, ..., 0.70684542, 0.56478249,
        1.        ]])

In [45]:
# Save df and similarity to pickle file

pickle.dump(df[['product_id', 'product_name']], open('products.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl','wb'))

In [46]:
product_index = training_df[training_df['product_name'] == 'Kiwi'].index[0]
product_index

29

In [47]:
similarity[product_index]

array([0.        , 0.        , 0.        , 0.06286946, 0.05698029,
       0.        , 0.07106691, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.06286946, 0.        ,
       0.38138504, 0.08703883, 0.08703883, 0.08703883, 0.09090909,
       0.10050378, 0.1672484 , 0.10050378, 0.09534626, 0.34815531,
       0.18181818, 0.17407766, 0.09090909, 0.19069252, 1.        ,
       0.18181818, 0.09534626, 0.47673129, 0.16116459, 0.10050378,
       0.18181818, 0.36363636, 0.17407766, 0.19069252, 0.19069252,
       0.08703883, 0.09534626, 0.        , 0.        , 0.        ,
       0.06286946, 0.        , 0.06428243, 0.        , 0.        ,
       0.05698029, 0.        , 0.        , 0.        , 0.05598925,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.11605177, 0.        , 0.        , 0.07312724,
       0.        , 0.        , 0.05913124, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [48]:
sorted(similarity[product_index])

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.045980048987170286,
 0.047088160934801115,
 0.04828045495852676,
 0.052486388108147805,
 0.05415303610738824,
 0.05504818825631803,
 0.05504818825631803,
 0.05598925109558543,
 0.05598925

In [49]:
recommended_product_indexes = sorted(list(enumerate(similarity[product_index])), reverse=True, key=lambda x: x[1])[1:6]
recommended_product_indexes

[(32, 0.4767312946227962),
 (15, 0.38138503569823695),
 (36, 0.36363636363636365),
 (24, 0.3481553119113957),
 (183, 0.23652495839563306)]

In [50]:
for index in recommended_product_indexes:
    print(df.iloc[index[0]].product_name)

Mulberry
Apple
Red Onions
Green Bell Pepper
Tropical Earring
